In [215]:
from libpy import readfile

filename = "2024/day15/input/sample.txt"
#filename = "2024/day15/input/sample2.txt"
#filename = "2024/day15/input/sample3.txt"
filename = "2024/day15/input/input.txt"
lines = readfile.lines(filename)

map = []
instructions = ""
map_finished = False
for line in lines:
    if line == "":
        map_finished = True
    elif map_finished:
        instructions += line
    else:
        map.append(line)
        
width = len(map[0])
height = len(map)

walls = []
boxes = []
robot = None
for y in range(height):
    for x in range(width):
        if map[y][x] == "#":
            walls.append((x, y))
        elif map[y][x] == "O":
            boxes.append((x, y))
        elif map[y][x] == "@":
            robot = (x, y)

In [216]:
class Map:
    def __init__(self, boxes, walls, robot):
        self.boxes = boxes
        self.walls = walls
        self.robot = robot
    
    def is_wall(self, x, y):
        return (x, y) in self.walls
    
    def hit_test(self, x, y):
        return [b for b in self.boxes if b.hit_test(x,y)]
    
    def move(self, dx, dy):
        new_robot = (self.robot[0] + dx, self.robot[1] + dy)
        if not self.is_wall(new_robot[0], new_robot[1]):
            robot_hits_box = [b for b in self.boxes if b.hit_test(new_robot[0], new_robot[1])]
            if len(robot_hits_box) == 1:
                if robot_hits_box[0].move_if_possible(self, dx, dy):
                    self.robot = new_robot
            else:
                self.robot = new_robot
    
    def execute_instructions(self, instructions):
        for instruction in instructions:
            match instruction:
                case "<": self.move(-1, 0)
                case "^": self.move(0, -1)
                case ">": self.move(1, 0)
                case "v": self.move(0, 1)
    
    def get_score(self):
        score = 0
        for b in self.boxes:
            score += 100 * b.y + b.x
        return score
    
    def __str__(self):
        width = max([w[0] for w in self.walls]) + 1
        height = max([w[1] for w in self.walls]) + 1

        result = ""
        for y in range(height):
            for x in range(width):
                boxes_on_position = self.hit_test([(x, y)])
                if self.is_wall(x, y):
                    result += "#"
                elif len(boxes_on_position) == 1:
                    result += "[" if boxes_on_position.pop().x == x else "]"
                elif self.robot == (x, y):
                    result += "@"
                else:
                    result += "."
            result += "\n"
        return result
        
class Box:
    def __init__(self, x, y, width = 1):
        self.x = x
        self.y = y
        self.width = width
        
    def hit_test(self, x, y):
        return self.y == y and x >= self.x and x <= self.x + self.width - 1
    
    def move_if_possible(self, map, dx, dy):
        cm = self.can_move(map, dx, dy)
        if cm[0]:
            for box in cm[1]:
                box.move(dx, dy)
            self.move(dx, dy)
            return True
        
        return False
    
    def move(self, dx, dy):
        self.x += dx
        self.y += dy
    
    def can_move(self, map, dx, dy):
        new_x = self.x + dx
        new_y = self.y + dy
        
        boxes_to_move = set()
        for x in range(new_x, new_x + self.width):
            if map.is_wall(x, new_y):
                return False, None
                
            for box in map.hit_test(x, new_y):
                if box == self or box in boxes_to_move:
                    continue
                bcm = box.can_move(map, dx, dy)
                if not bcm[0]:
                    return False, None
                boxes_to_move.add(box)
                for b in bcm[1]:
                    boxes_to_move.add(b)
                        
        return True, boxes_to_move

Part A

In [ ]:
normal_boxes = [Box(b[0], b[1], 1) for b in boxes]

map = Map(normal_boxes, walls, robot)
map.execute_instructions(instructions)

print(map.get_score())

Part B

In [ ]:
wide_walls = [(w[0] * 2, w[1]) for w in walls] + [(w[0] * 2 + 1, w[1]) for w in walls]
wide_boxes = [Box(b[0] * 2, b[1], 2) for b in boxes]

map = Map(wide_boxes, wide_walls, (robot[0] * 2, robot[1]))
map.execute_instructions(instructions)

print(map.get_score())